In [26]:
from typing import List
import math
from functools import cmp_to_key

class Point: 
    def __init__(self, x, y): 
        self.x = x 
        self.y = y 
        
    def distance_to_line(self, p1, p2):
        x_diff = p2.x - p1.x
        y_diff = p2.y - p1.y
        P_x = self.x - p1.x
        P_y = self.y - p1.y
        num = abs(y_diff*P_x - x_diff*P_y )
        den = math.sqrt(y_diff**2 + x_diff**2)
        return num / den
    
    def subtract(self, p):
        return Point(self.x - p.x, self.y - p.y)
    
def cross_product(p1, p2):
    return p1.x * p2.y - p2.x * p1.y

def direction(p1, p2, p3):
    return  cross_product(p3.subtract(p1), p2.subtract(p1))

def distance_sq(p1, p0):
    
    return (p1.x-p0.x)**2 + (p1.y-p0.y)**2
        
# find the point with minimum y coordinate
# in case of tie choose the point with minimun x-coordinate
def find_min_y(points):
    miny = 999999
    mini = 0
    for i, point in enumerate(points):
        if point.y < miny:
            miny = point.y
            mini = i
        if point.y == miny:
            if point.x < points[mini].x:
                mini = i
    return points[mini], mini

# comparator for the sorting 
def polar_comparator(p1, p2, p0):
    d = direction(p0, p1, p2)
    if d < 0:
        return -1
    if d > 0:
        return 1
    if d == 0:
        if distance_sq(p1, p0) < distance_sq(p2, p0):
            return -1
        else:
            return 1

def graham_scan(points):
    # let p0 be the point with minimum y-coordinate,
    # or the leftmost such point in case of a tie
    p0, index = find_min_y(points)

    # swap p[0] with p[index]
    points[0], points[index] = points[index], points[0]

    # sort the points (except p0) according to the polar angle
    # made by the line segment with x-axis in anti-clockwise direction
    sorted_polar = sorted(points[1:], key=cmp_to_key ( lambda p1, p2: polar_comparator(p1, p2, p0)))
    
    
    # if more than two points are collinear with p0, keep the farthest
    to_remove = []
    for i in range(len(sorted_polar) - 1):
        d = direction(sorted_polar[i], sorted_polar[i + 1], p0)
        if d == 0:
            to_remove.append(i)
    sorted_polar = [i for j, i in enumerate(sorted_polar) if j not in to_remove]

   
    m = len(sorted_polar)
    if m < 2:
        return 0

    else:
        stack = []
        stack_size = 0
        stack.append(points[0])
        stack.append(sorted_polar[0])
        stack.append(sorted_polar[1])
        stack_size = 3

        for i in range(2, m):
            while (True):
                d = direction(stack[stack_size - 2], stack[stack_size - 1], sorted_polar[i])
                if d < 0: # if it makes left turn
                    break
                else: # if it makes non left turn
                    stack.pop()
                    stack_size -= 1
            stack.append(sorted_polar[i])
            stack_size += 1
    return stack


        
class Airport:
    def airport(self, houses: List[List[int]]) -> float:
        """
        Find the best place to build airport and
        calculate the average distance from all the house to airport

        Parameters:
            houses(list[list[int]]): List of houses.
                Each house contains [x,y] coordination.

        Returns:
            distance(float)
        """
        points=[]
        for i in houses:
            points.append(Point(i[0],i[1]))
        ans = graham_scan(points)
    
        
        
        if ans==0:
            return 0
        
        
        min_distance=9999999999999
        
        for i in range(len(ans)-1):
     
            total_distance=0
                
            for h in points:

                current_distance = h.distance_to_line(ans[i],ans[i+1])
                total_distance+=current_distance

            average_distance =  total_distance / len(points)

            if average_distance < min_distance :
                min_distance=average_distance
        
        
        #line between last point and start point 
        total_distance=0
        for h in points:

            current_distance = h.distance_to_line(ans[0],ans[-1])
            total_distance+=current_distance

        average_distance =  total_distance / len(points)

        if average_distance < min_distance :
            min_distance=average_distance
        
        
        return min_distance

 

In [27]:
ans=Airport().airport([[1,1],[2,2],[0,2],[2,0],[2,4],[3,3],[4,2],[4,1],[4,0]])

In [28]:
ans

1.3356461422412562

In [29]:
if __name__ == "__main__":
    print(Airport().airport([[0,0],[1,0]]))
    """
    0.0
    """
    print(Airport().airport([[0,0],[1,0],[0,1]]))
    """
    *.
    **
    # Convex: [[0, 0], [1, 0], [0, 1]]
    0.2357022603955159
    """
    print(Airport().airport([[0,0],[2,0],[0,2],[1,1],[2,2]]))
    """
    *.*
    .*.
    *.*
    # Convex: [[0, 0], [2, 0], [2, 2], [0, 2]]
    1.0
    """
    print(Airport().airport([[1,1],[2,2],[0,2],[2,0],[2,4],[3,3],[4,2],[4,1],[4,0]]))
    """
    ..*..
    ...*.
    *.*.*
    .*..*
    ..*.*
    # Convex: [[0, 2], [2, 0], [4, 0], [4, 2], [2, 4]]
    1.3356461422412562
    """

0
0.2357022603955158
1.0
1.3356461422412562
